In [8]:
!pip install -q accelerate PyPDF2 faiss-cpu nltk torch transformers gradio


In [9]:
!pip install langchain-google-genai


In [10]:
!pip install openai langchain gradio nltk faiss-cpu transformers


In [11]:
!pip install langchain_community


In [1]:
!pip install --upgrade langchain openai transformers faiss-cpu



In [ ]:
import gradio as gr
import nltk
import faiss
import pandas as pd
from transformers import AutoModel
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

# Function to load models and resources
def load_resources():
    nltk.download('punkt')  # Ensure the Punkt tokenizer is downloaded
    # Load the embedding model
    model = AutoModel.from_pretrained('jinaai/jina-embeddings-v2-base-en', trust_remote_code=True)
    # Load OpenAI LLM
    llm = ChatOpenAI(
        model="gpt-4",  # or "gpt-3.5-turbo"
        openai_api_key="",
        temperature=0.8
    )
    # Load text data and FAISS index
    df = pd.read_csv('med_assistant_db.csv')
    sentences = df['text'].tolist()
    faiss_index = faiss.read_index('med_assistant_index')
    return model, llm, sentences, faiss_index

# Initialize resources
model, llm, sentences, faiss_index = load_resources()

# Function to search in the FAISS index
def search_in_index(index, query, sentences, model, top_k=5, threshold=0.5):
    # Generate embeddings for the query
    query_embedding = model.encode([query], convert_to_tensor=True).cpu().detach().numpy()
    # Perform FAISS search
    distances, indices = index.search(query_embedding, top_k)
    # Retrieve sentences and distances
    results = [(sentences[i], distances[0][j]) for j, i in enumerate(indices[0])]
    # Filter results based on a relevance threshold
    filtered_results = [res for res in results if res[1] < threshold]
    return filtered_results

# Function to craft the prompt and generate results
def craft_prompt_generate_results(input_str):
    # Refined prompt for better results
    template = """
    We are conducting an analysis to assist medical professionals by analyzing patient texts. This analysis is meant to complement, not replace, professional medical advice.

    If the input text does not indicate any concerning symptoms, state "No concerning symptoms identified," and do not include potential disorders or symptoms unless explicitly mentioned.

    Input Provided:
    {input_str}

    Your analysis should include:
    - Assessment: Clearly indicate whether there are concerning symptoms or not.
    - (Optional) Potential Disorders: Only list if specific symptoms indicate a disorder.
    - (Optional) Symptoms: Only list symptoms explicitly mentioned or implied in the text.
    - Recommendations: Provide general health recommendations based on the input, or state "No specific recommendations."

    Format your analysis in a clear, structured manner.
    """
    prompt_template = PromptTemplate(input_variables=["input_str"], template=template)
    question_chain = LLMChain(llm=llm, prompt=prompt_template)
    overall_chain = SimpleSequentialChain(chains=[question_chain])
    # Run the chain with the input string
    result = overall_chain.run(input_str)
    return result

# Function to clean up the output from the LLM
def post_process_output(output):
    if "No concerning symptoms identified" in output:
        # Remove unnecessary sections for this case
        if "- Potential Disorders:" in output:
            output = output.split("- Potential Disorders:")[0].strip()
    return output

# Gradio interface functions
def analyze_symptoms(symptoms):
    # Search in the FAISS index for related sentences
    results = search_in_index(faiss_index, symptoms, sentences, model)
    # Combine retrieved sentences into a single text
    text = ' '.join([sentence for sentence, _ in results])
    # Prepare input for the LLM
    input_str = f"query: {symptoms}\nEnd of Query\ntext: {text}"
    # Generate results using the LLM
    result = craft_prompt_generate_results(input_str)
    # Post-process the LLM output
    processed_result = post_process_output(result)
    return processed_result

# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# AI Med Assistant [Experiment]")
    gr.Markdown(
        "This tool assists in analyzing patient symptoms using OpenAI's GPT models. Please input symptoms to get a preliminary analysis."
    )
    # Input field for symptoms
    input_text = gr.Textbox(label="Enter Symptoms", placeholder="Type symptoms here...", lines=3)
    # Output field for analysis
    output_text = gr.Textbox(label="Analysis Result", placeholder="Results will appear here...", lines=10)
    # Button to trigger analysis
    analyze_button = gr.Button("Analyze Symptoms")
    # Link button to the analyze_symptoms function
    analyze_button.click(analyze_symptoms, inputs=input_text, outputs=output_text)

# Launch Gradio App
demo.launch()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
<ipython-input-2-a9f010df4f58>:16: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import Cha

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d3f16b55e96140afc8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [17]:
import faiss
index = faiss.read_index('med_assistant_index')
print("Index Loaded:", index.is_trained)


Index Loaded: True


In [18]:
from transformers import AutoModel
model = AutoModel.from_pretrained('jinaai/jina-embeddings-v2-base-en', trust_remote_code=True)
print("Model Loaded Successfully")


Model Loaded Successfully


In [19]:
query = "My joints feel stiff and painful, especially in the morning."
query_embedding = model.encode([query], convert_to_tensor=True).cpu().detach().numpy()
distances, indices = index.search(query_embedding, k=5)
print("Search Results:", distances, indices)


Search Results: [[35.4064   41.269592 47.384407 60.73436  66.36656 ]] [[22340 21909 21928 21927 21881]]


In [ ]:
import openai

# Test OpenAI API connection
try:
    openai.api_key = ""
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt="Test prompt",
        max_tokens=10
    )
    print("OpenAI API Working!")
except Exception as e:
    print(f"OpenAI API Error: {e}")


OpenAI API Error: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742

